## Spark streaming test

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from reddit.reddit_streaming import *

creds, config = read_files()
subreddit = config["subreddit"]
kafka_host = config["kafka_host"]
spark_host = config["spark_host"]
aws_client = creds["aws-client"]
aws_secret = creds["aws-secret"]

print("imported modules")

: 

In [4]:
try:
    main()

except KeyboardInterrupt:
    print("stopped spark streaming.")
    
except Exception as e:
    print(e)

22/05/17 21:38:13 WARN SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.lang.reflect.Constructor.newInstance(Constructor.java:423)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.ConstructorCommand.execute(ConstructorCommand.java:69)
py4j.ClientServ

created spark successfully


22/05/17 21:38:28 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
22/05/17 21:38:30 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
22/05/17 21:39:14 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
22/05/17 21:39:15 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-ffe4e3a2-4241-4e0f-8154-d3f89e10c628--1325427250-driver-0-1, groupId=spark-kafka-source-ffe4e3a2-4241-4e0f-8154-d3f89e10c628--1325427250-driver-0] Error while fetching metadata with correlation id 2 : {reddit_technology=LEADER_NOT_AVAILABLE}
22/05/17 21:39:15 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-source-ffe4e3a2-4241-4e0f-8154-d3f89e10c628--1325427250-driver-0-1, groupId=spark-kafka-source-ffe4e3a2-4241-4e0f-8154-d3f89e10c

An error occurred while calling o174.awaitTermination


# Check dataset

In [5]:
try:
    spark, sc = init_spark()
    test = spark.read.format("delta").option("header", True).load("s3a://reddit-stevenhurwitt/" + subreddit)
    test_pandas = test.toPandas()
    print("read delta table to df.")

except KeyboardInterrupt:
    print("loading data took too long... cancelled.")

created spark successfully


In [6]:
test_pandas

,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,subreddit_name_prefixed,...,author_flair_text_color,permalink,parent_whitelist_status,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video
0,NaN,technology,,t2_hqstdwu0,False,None,0,False,Elon Musk is using a 'dog ate the homework' ex...,r/technology,...,None,/r/technology/comments/uqwpba/elon_musk_is_usi...,all_ads,False,https://markets.businessinsider.com/news/stock...,11994694,1.652710e+09,0,None,False
1,NaN,technology,,t2_7ccf,False,None,0,False,Royal Mail drone fleet takes to the air for re...,r/technology,...,None,/r/technology/comments/uo0r18/royal_mail_drone...,all_ads,False,https://www.edinburghnews.scotsman.com/news/pe...,11965116,1.652361e+09,0,None,False
2,NaN,technology,,t2_7ccf,False,None,0,False,These Nanobots Can Swim Around a Wound and Kil...,r/technology,...,None,/r/technology/comments/uow5as/these_nanobots_c...,all_ads,False,https://www.wired.com/story/these-nanobots-can...,11973872,1.652461e+09,0,None,False
3,NaN,technology,,t2_6utha,False,None,0,False,Did Twitch Violate Texas’ Social Media Law By ...,r/technology,...,None,/r/technology/comments/ur2490/did_twitch_viola...,all_ads,False,https://www.techdirt.com/2022/05/16/did-twitch...,11996188,1.652724e+09,0,None,False
4,NaN,technology,,t2_lz41dbnl,False,None,0,False,"real. fucking. conservative. civil liberties, ...",r/technology,...,None,/r/technology/comments/uo8w92/real_fucking_con...,all_ads,False,https://arstechnica.com/tech-policy/2022/05/te...,11967337,1.652383e+09,0,None,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,NaN,technology,,t2_83wk7s3r,False,None,0,False,Novel Diamond Semiconductors Operate at Highes...,r/technology,...,None,/r/technology/comments/uqwzkm/novel_diamond_se...,all_ads,False,https://www.ad-na.com/magazine_en/archives/44,11994761,1.652711e+09,0,None,False
74,NaN,technology,,t2_1argepoh,False,None,0,False,Apple iPod creator warns the metaverse will en...,r/technology,...,None,/r/technology/comments/uo6vg8/apple_ipod_creat...,all_ads,False,https://www.bbc.com/news/business-61423268,11966776,1.652378e+09,0,None,False
75,NaN,technology,,t2_a5jwjm1s,False,None,0,False,A colony of blue-green algae can power a compu...,r/technology,...,None,/r/technology/comments/uo75rs/a_colony_of_blue...,all_ads,False,https://interestingengineering.com/blue-green-...,11966852,1.652379e+09,0,None,False
76,NaN,technology,,t2_bf38q9nm,False,None,0,False,"Musk, Twitter CEO spar over bot accounts",r/technology,...,None,/r/technology/comments/ur2j9k/musk_twitter_ceo...,all_ads,False,https://thehill.com/policy/technology/,11996262,1.652726e+09,0,None,False


## Debug main function

In [19]:
spark, sc = init_spark()

created spark successfully


In [5]:
stage_df = read_kafka_stream(spark, sc)

In [4]:
try:
    write_stream(stage_df)

except KeyboardInterrupt:
    print("stopped streaming.")

22/05/11 18:10:17 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
22/05/11 18:10:23 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
22/05/11 18:12:30 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 6000 milliseconds, but spent 12156 milliseconds


stopped streaming.


In [5]:
spark.stop()
print("stopped spark successfully.")

NameError: name 'spark' is not defined